## 模型融合

### 载入数据

这个数据集是关于匹马印第安人糖尿病的数据集，包含了以下几个特征：

1. preg：怀孕次数。
2. plas：口服葡萄糖耐量试验中2小时后的血浆葡萄糖浓度。
3. pres：舒张压（mm Hg）。
4. skin：肱三头肌皮肤褶皱厚度（mm），这是一个用于估计体脂百分比的指标。
5. test：2小时血清胰岛素（mu U/ml）。
6. mass：体重指数（体重（kg）/ 身高（m）^2）。
7. pedi：糖尿病家族史函数，这是一个考虑了糖尿病和家族关系的函数。
8. age：年龄（岁）。
9. class：类标签，0表示无糖尿病，1表示有糖尿病。

这个数据集通常用于糖尿病的预测任务，例如我们可以使用这些特征来预测一个人是否有糖尿病。为了更好地理解这个数据集，我们可以进行一些描述性统计分析，例如计算每个特征的均值、中位数、标准差等，或者绘制每个特征的直方图和箱线图等。

In [1]:
import pandas

In [2]:
data = "data/pima-indians-diabetes.data.csv"
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
df = pandas.read_csv(data, names=names)

In [3]:
df.head()

,preg,plas,pres,skin,test,mass,pedi,age,class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
df['class'].unique()

array([1, 0])

### 投票器模型融合

使用 VotingClassifier 类来实现投票器模型融合。

投票器模型融合是一种简单的模型融合方法，它结合了多个模型的预测结果，通过投票的方式来做出最终的预测。

In [5]:
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
import warnings
warnings.filterwarnings('ignore')

In [6]:
df.head(2)

,preg,plas,pres,skin,test,mass,pedi,age,class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0


In [7]:
array = df.values

In [8]:
array

array([[  6.   , 148.   ,  72.   , ...,   0.627,  50.   ,   1.   ],
       [  1.   ,  85.   ,  66.   , ...,   0.351,  31.   ,   0.   ],
       [  8.   , 183.   ,  64.   , ...,   0.672,  32.   ,   1.   ],
       ...,
       [  5.   , 121.   ,  72.   , ...,   0.245,  30.   ,   0.   ],
       [  1.   , 126.   ,  60.   , ...,   0.349,  47.   ,   1.   ],
       [  1.   ,  93.   ,  70.   , ...,   0.315,  23.   ,   0.   ]])

In [9]:
X = array[:,0:8]
Y = array[:,8]
kfold = model_selection.KFold(n_splits=5, shuffle=True)

# 创建投票器的子模型
estimators = []
model_1 = LogisticRegression()
estimators.append(('logistic', model_1))

model_2 = DecisionTreeClassifier()
estimators.append(('dt', model_2))

model_3 = SVC()
estimators.append(('svm', model_3))

# 构建投票器融合
ensemble = VotingClassifier(estimators)
result = model_selection.cross_val_score(ensemble, X, Y, cv=kfold)
print(result.mean())

0.774705033528563


- 投票器模型融合的优点是可以结合多个模型的优点，提高模型的稳定性和泛化能力。
- 此外，它还可以处理模型之间的不确定性，因为它不依赖于任何一个单独的模型。
- 然而，它的缺点是不能处理模型之间的依赖性，因为它假设每个模型都是独立的。

- 投票器模型融合通常在我们有多个模型，且每个模型的性能需要使用时是好用的，例如在比赛或者项目中，我们可能会训练多个模型，然后使用投票器模型融合来结合这些模型的预测结果。

### Bagging

- Bagging（Bootstrap Aggregating）是一种模型融合方法，它通过自助采样（Bootstrap Sampling）生成多个不同的训练集，
- 然后训练多个模型，最后通过平均（对于回归问题）或者投票（对于分类问题）来做出最终的预测。

In [10]:
from sklearn.ensemble import BaggingClassifier

In [11]:
dt = DecisionTreeClassifier()
num = 100
kfold = model_selection.KFold(n_splits=5, shuffle=True)
model = BaggingClassifier(base_estimator=dt, n_estimators=num, random_state=2018)
result = model_selection.cross_val_score(model, X, Y, cv=kfold)
print(result.mean())

0.7550887021475258


- Bagging的优点是可以通过自助采样增加模型的多样性，从而提高模型的稳定性和泛化能力。
- 此外，由于每个模型都是独立训练的，所以Bagging可以并行化，从而提高计算效率。
- 然而，Bagging的缺点是它不能处理模型之间的依赖性，因为它假设每个模型都是独立的。
- Bagging通常在我们有一个容易过拟合的模型，例如决策树或者神经网络，时使用，因为Bagging可以通过增加模型的多样性来防止过拟合。

### RandomForest

使用RandomForestClassifier类来创建一个随机森林模型。

- 随机森林是一种模型融合方法，它结合了Bagging和随机子空间方法，通过自助采样生成多个不同的训练集，
- 然后对每个训练集训练一个决策树模型，同时在每个决策树的训练过程中，随机选择一部分特征进行分裂，最后通过平均（对于回归问题）或者投票（对于分类问题）来做出最终的预测。


In [12]:
from sklearn.ensemble import RandomForestClassifier
num_trees = 100
max_feature_num = 5
kfold = model_selection.KFold(n_splits=5, shuffle=True)
model = RandomForestClassifier(n_estimators=num_trees, max_features=max_feature_num)
result = model_selection.cross_val_score(model, X, Y, cv=kfold)
print(result.mean())

0.765571683218742


- 随机森林的优点是可以通过自助采样和随机子空间方法增加模型的多样性，从而提高模型的稳定性和泛化能力。
- 此外，由于每个模型都是独立训练的，所以随机森林可以并行化，从而提高计算效率。
- 然而，随机森林的缺点是它不能处理模型之间的依赖性，因为它假设每个模型都是独立的。
- 随机森林通常在我们有一个容易过拟合的模型，例如决策树时使用，因为随机森林可以通过增加模型的多样性来防止过拟合。此外，随机森林还可以用于特征选择，因为它可以计算每个特征的重要性。

### Adaboost


- AdaBoost（Adaptive Boosting）是一种模型融合方法，它通过在每一轮中增加之前轮次预测错误的样本的权重，
- 然后训练一个新的模型，最后通过加权平均（对于回归问题）或者加权投票（对于分类问题）来做出最终的预测。

In [13]:
from sklearn.ensemble import AdaBoostClassifier
num_trees = 25
kfold = model_selection.KFold(n_splits=5, shuffle=True)
model = AdaBoostClassifier(n_estimators=num_trees, random_state=2018)
result = model_selection.cross_val_score(model, X, Y, cv=kfold)
print(result.mean())

0.7603344368050251


- AdaBoost的优点是可以通过逐步增加预测错误的样本的权重来提高模型的性能，特别是对于那些难以预测的样本。
- 此外，AdaBoost也可以用于特征选择，因为它可以计算每个特征的重要性。然而，AdaBoost的缺点是它比较敏感于噪声和异常值，因为它会不断增加这些样本的权重。

- AdaBoost通常在我们有一个偏差较大的模型，例如决策树桩（只有一层的决策树）时使用，因为AdaBoost可以通过增加模型的复杂性来减少偏差。此外，AdaBoost也可以用于特征选择，因为它可以计算每个特征的重要性。